In [2]:
import pandas as pd
import pickle as pk
import constants.constants as constants
import prompts.prompt_classifier as prompt_classifier
import prompts.prompt_banking as prompt_banking
import prompts.prompt_cards as prompt_card
import prompts.prompt_oeffnungszeiten as prompt_oeffnungszeiten
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.tools import tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType, Tool
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory

In [3]:
with open("./database/pd_frame.txt","rb") as file:
    pd_DF = pk.load(file)

In [4]:
chat = ChatOpenAI(openai_api_key=constants.OPENAI_API_KEY)

@tool
def card_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren von Bankomat-, Debit- und Kreditkarten."""
    messages = [
        SystemMessage(content=prompt_card.prompt_cards),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)


@tool
def banking_information(question):
    """Beantwortet Fragen zu dem Thema: Sperren des Online-Bankings."""
    messages = [
        SystemMessage(content=prompt_banking.prompt_banking),
        HumanMessage(content=question)
    ]

    return chat.predict_messages(messages)


@tool
def information_regarding_branches(question):
    """Beantwortet Fragen zu dem Thema: Bankfilialen (Öffnungszeiten, Ausstattung, etc.)"""
    df_context = "Basierend auf dem Schema des DataFrames 'df' beantworte die folgende Frage so gut du kannst, beachte dass Filialnamen nicht vollständig sind und gib alle Zeiten an:"
    agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model=constants.MODEL,openai_api_key=constants.OPENAI_API_KEY),
    pd_DF,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    )
    return agent.run(df_context+question)

c:\Users\North\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [5]:
tools = [card_information, banking_information, information_regarding_branches]

In [1]:
agent_kwargs = {
    "extra_prompt_messages": [MessagesPlaceholder(variable_name="memory")],
}
memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

memory.chat_memory.add_message(SystemMessage(content=prompt_classifier.system_message))
memory.chat_memory.add_message(first_ai_msg := AIMessage(content="Hier ist der Chatbot der Raiffeisenlandesbank Niederösterreich-Wien. Gerne beantworte ich Ihre Fragen zur Sperre von Karten sowie zur Ausstattung von Filialen."))

#print(first_ai_msg.content)

agent = initialize_agent(
    tools,
    chat,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    agent_kwargs=agent_kwargs,
    memory=memory,
)

NameError: name 'MessagesPlaceholder' is not defined

In [7]:
# while True:
#     msg = str(input("Your question: "))
#     run_context= """Regel 1: Beim Beantworten aller folgenden Fragen, merke dir, wenn dir wiederholt die selbe Antwort gestellt wird und weise darauf hin.
#     Regel 2: Du musst! Fragen beantworten zu denen du auch die Information hast, du kannst nicht einfach auf die Website referenzieren!"""
#     print(agent.run(run_context+msg))
#     #print(memory)

#     #print(memory.chat_memory)

#     if len(memory.chat_memory.messages) > 4:
#         del memory.chat_memory.messages[2]
#         del memory.chat_memory.messages[2]